In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [4]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [5]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect = True)

In [6]:
# View all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [7]:
# Save references to each table
measurement = Base.classes.measurement
station = Base.classes.station

In [8]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [9]:
# Find the most recent date in the data set.

In [11]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 
latest_date = session.query(measurement.date).order_by(measurement.date.desc()).first()[0]
latest_date

'2017-08-23'

In [45]:
results = session.query(measurement.date, func.avg(measurement.prcp)).filter(measurement.date>=query_date).group_by(measurement.date).all()
results_data = {}
for date, prcp in results:
    results_data{"date"} = prcp

results_data

SyntaxError: invalid syntax (<ipython-input-45-a1e4bf647e6a>, line 4)

In [23]:
# Calculate the date one year from the last date in data set.
query_date = dt.date(2017, 8, 23) - dt.timedelta(days=365)
query_date = query_date.strftime("%Y-%m-%d")
query_date

'2016-08-23'

In [ ]:
# Perform a query to retrieve the data and precipitation scores
precip_data = session.query(measurement.date, measurement.prcp).filter(measurement.date >= query_date).order_by(measurement.date).all()
precip_data

In [ ]:
# Save the query results as a Pandas DataFrame and set the index to the date column
df = pd.DataFrame(precip_data, columns = ["date", "prcp"])
df = df.set_index("date")
df

In [ ]:
# Sort the dataframe by date
plot_df = df.sort_values(by = "date")

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
line_chart = plot_df.plot(kind = "line")
plt.xticks(rotation = "90")
plt.xlabel("Date")
plt.ylabel("Inches")

plt.show()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
summary = df.describe()
summary["prcp"] = summary["prcp"].map('{:.2f}'.format)
summary

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
total_stations = session.query(station.station).count()
total_stations

In [ ]:
# Design a qus (i.e. what stations have the most rows?)
# List the stations and the countery to find the most active stations in descending order.
active_stations = session.query(func.count(measurement.station), measurement.station).group_by(measurement.station).order_by(func.count(measurement.station).desc()).all()
active_stations

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
most_active = session.query(func.min(measurement.tobs), func.max(measurement.tobs), func.avg(measurement.tobs)).filter(measurement.station == "USC00519281").all()
print(f'The lowest temperature is {most_active[0][0]} degrees')
print(f'The hightest temperature is {most_active[0][1]} degrees')
print(f'The average temperature is {most_active[0][2]} degrees')

In [ ]:
# Using the most active station id
tobs_filter = session.query(measurement.station, func.count(measurement.station)).filter(measurement.date >= query_date).group_by(measurement.station).order_by(func.count(measurement.station).desc()).all()
tobs_filter

In [ ]:
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram

# Last 12 months of data for all sations
tobs_query = session.query(measurement.tobs).filter(measurement.date >= query_date).all()
df = pd.DataFrame(tobs_query, columns = ["tobs"])
plt.hist(df, bins = 12, label = "tobs")
plt.xlabel("Temperature")
plt.ylabel("Frequency")
plt.legend(loc = "upper right")

plt.show()

In [ ]:
# Last 12 months of data for station with highest number of observations

tobs_query = session.query(measurement.tobs).filter(measurement.date >= query_date).filter(measurement.station == "USC00519397").all()
df = pd.DataFrame(tobs_query, columns = ["tobs"])
plt.hist(df, bins = 12, label = "tobs")
plt.xlabel("Temperature")
plt.ylabel("Frequency")
plt.legend(loc = "upper right")

plt.show()

# Close session

In [ ]:
# Close Session
session.close()